In [1]:
import asyncio
import nest_asyncio
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from IPython.display import clear_output



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# Carga y divide los documentos
loader = WebBaseLoader("https://cgsait.udg.mx/es/cads#:~:text=LEO%20%C3%81TROX%2C%20superc%C3%B3mputo%20a%20tu,poder%20de%20procesamiento%20de%20datos.&text=150%20nodos%20c%C3%B3mputo%2C%20con%20un,c%C3%B3mputo%20con%20tecnolog%C3%ADa%20XEON%20PHI%20.")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

# Configuración de embeddings y vectorstore
local_embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")
vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

In [3]:
# Configuración del modelo de lenguaje y plantilla de RAG
model = ChatOllama(model="smollm2:latest")  # Cambiado a llama2 por compatibilidad

RAG_TEMPLATE = """
Eres un asiste de preguntas y respuestas. Usa la informacion recuperada para contestar las preguntas, relacionadas al CADS. Si no sabes la respuesta, solo di que no conoces la respuesta. Usa tres sentencias como maximo para dar una respuesta concisa.

<context>
{context}
</context>

Contesta la siguiente pregunta: 

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

# Definir la función para formatear documentos
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever = vectorstore.as_retriever()

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

# Función para manejar una sola pregunta
async def process_question(question):
    print(f"Procesando pregunta: {question}\n")
    response = ""
    try:
        async for chunk in chain.astream(question):
            response += chunk
            clear_output(wait=True)
            print(f"Pregunta: {question}")
            print(f"Respuesta: {response}", flush=True)
        return response
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

# Función para ejecutar la pregunta
def run_single_question(question):
    return asyncio.get_event_loop().run_until_complete(process_question(question))

In [4]:
respuesta=run_single_question("Cual es la mision del CADS")

RuntimeError: This event loop is already running